In [ ]:
'''
In this notebook we're simply going to assume a function, then sample few points from it, then discard the function parameters. Then will try to find the parameters back from sampled points. In the process will try to learn error surfaces and how a simple gradient descent works.
'''